In [3]:
    # !pip install textblob 
# !pip install scikit-learn
# !pip install gensim
# !pip install nltk
# !pip install seaborn
# !pip install matplotlib
# !pip install skfeature-chappers



In [4]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection
from textblob import TextBlob 
import numpy as np
import re
import nltk
nltk.download('stopwords')

from gensim.parsing.porter import PorterStemmer
from gensim.utils import tokenize

from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')]
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/suleman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:

def listing():
    """
        i) Read data from csv.
        ii) Convert data to numpy array.
        iii) Split data with comma.
        iv) Appends data into list.
    """

    url = "Data/all_brands_export.csv"
    # url = "Data/acer-phones_export.csv"
    db=pd.read_csv(url,low_memory = False)#,sep= ',', error_bad_lines=False, index_col=False, dtype='unicode')
    # db["review"]
    reviews = np.array(db["review"])
    # reviews = np.array(["visually device is fair but if we go with cell durations that is below mark."])
    # print(reviews)

    super_list = []
    # sentece=["visually device is fair but if we go with cell durations that is below mark."]
    for sent in reviews:

        temp= []
        try:
            temp = list(sent.split(","))
            super_list.append(temp)

        except:
            super_list.append("None")
    return super_list


In [6]:
def pre_process(sentence):  
    """
        i) Remove special characters.
        ii) Remove single characters.
        iii) Replace multiple spaces with single spaces.
    """


    # Remove all the special characters
    processed_sentence = re.sub(r'\W', ' ', str(sentence))

    # remove all single characters
    processed_sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_sentence)

    # Remove single characters from the start
    processed_sentence= re.sub(r'\^[a-zA-Z]\s+', ' ', processed_sentence) 

    # Substituting multiple spaces with single space
    processed_sentence= re.sub(r'\s+', ' ', processed_sentence, flags=re.I)

    # Removing prefixed 'b'
    processed_sentence = re.sub(r'^b\s+', '', processed_sentence)
    
   
    return processed_sentence


In [7]:
def updated_sentence(sentence):
    """
        i) Lower cases the sentence.
        ii) Sentence stemming.
        iii) Tokenization of sentence.
        iv) Stop-words removal.
        v) Returns list after all above operations.
    """
    sentence = sentence.lower()
    new = " "
    p = PorterStemmer()
    processed = pre_process(sentence)
    stemmed = p.stem(processed)
    tokens = tokenize(stemmed.strip())
    lst=list(tokens)
    lst2 = []
    for i in lst:
        #using nltk dic for stopwords
        if  i not in stopwords :
            lst2.append(i)
    return (new.join(lst2))

In [8]:
def polarity_scaling(polarity):
    """
        Scaling of polarity into three classes i.e. negative, positive and neutral.
    """
    # 0  nutral
    if polarity  == 0:
        return 0
    # greater then 0  positive
    elif polarity > 0:
        return 1
    else:
    # less then 0  negative
        return -1    

In [9]:

def reviews_list_polarity(super_list):
    p = PorterStemmer()

    features_dictionary={
        "battery": ["cell","power","battery","charge","recharge","cycle","volt","ampere","current","electric"],
        "display": ["display","screen","panel","brightness","glass","touch","graphics","resolution","refresh rate","color","colour","visuals"],
        "camera": ["camera","pixels","image","picture","zoom","selfie","optical","flash","record","hdr","panorama","night vision","time lapse","slow motion","slo-mo","fps","video"],
        "storage": ["storage","ram","gb","mb","tb","space","memory","cache"],
        # "price": ["price","cost","value","money","expensive","cheap","affordable"]
    }
    
    
    battery_polarity =[]
    display_polarity =[]
    camera_polarity =[]
    storage_polarity =[]
    # price_polarity =[]
    
    
    
    for_reviews = []
    for_polarity = []
    updated_review_Dict = {}
    super_list_2 = []
    list_reviews = []
    list_polarity = []

    for remove_none_type in super_list:
        if remove_none_type != "None":
            super_list_2.append(remove_none_type)

            lst_list = []
            lst_list_2 = []
            for i in super_list_2:
                new1 =i.strip().split(".")

            for j in new1:
                j = updated_sentence(j)
                new_sentence =j.replace("\r\n" ,"")
                for feature in features_dictionary:
                    for word in features_dictionary[feature]:
                        if p.stem(word) in new_sentence:
                        # if word in new_sentence:
                            pol = TextBlob(new_sentence).sentiment.polarity
                            polarity=polarity_scaling(pol)
                            tup = (new_sentence ,polarity)
                            lst_list.append(tup)

                            if  features_dictionary[feature] == features_dictionary["display"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                display_polarity.append(polarity)
                            # else:
                            #     camera_polarity.append(0)
                            if  features_dictionary[feature] == features_dictionary["camera"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                camera_polarity.append(polarity)
                            # else:
                            #     camera_polarity.append(0)

                            if  features_dictionary[feature] == features_dictionary["battery"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                battery_polarity.append(polarity)
                            # else:
                            #     battery_polarity.append(0)

                            if  features_dictionary[feature] == features_dictionary["storage"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                storage_polarity.append(polarity)
                            # else:
                            #     storage_polarity.append(0)
    
            for remove_tuple in lst_list:
                if remove_tuple != ('',0):
                    lst_list_2.append(remove_tuple)

            if lst_list_2 != []:
                for_reviews.append(lst_list_2[0][0])
                for_polarity.append(lst_list_2[0][1])
  
        updated_review_Dict["reviews"]=for_reviews
        updated_review_Dict["polarity"]=for_polarity

        
        list_reviews=updated_review_Dict["reviews"]
        list_polarity=updated_review_Dict["polarity"]
    return list_reviews,list_polarity ,battery_polarity ,display_polarity ,storage_polarity ,camera_polarity


In [10]:
def per_mobile_phone_review():
    review_list = listing()
    updated_review_list=[]
    for i in range(len(review_list)):
        updated_review_list.append(reviews_list_polarity(review_list[i]))
    return updated_review_list

In [13]:
#for changing the colume name 

data=pd.DataFrame(per_mobile_phone_review())
data.rename(columns = {0:'reviews', 1:'polarity',2:'battery_polarity',
                       3:'display_polarity',4:'storage_polarity',5:'camera_polarity'},
            inplace = True)
# data
for i in data["polarity"] :
    if i == []:
        i.append(0)
for i in data["battery_polarity"] :
    if i == []:
        i.append(0)
for i in data["display_polarity"] :
    if i == []:
        i.append(0)
        
for i in data["storage_polarity"] :
    if i == []:
        i.append(0)
for i in data["camera_polarity"] :
    if i == []:
        i.append(0)


In [14]:
data.head()

,reviews,polarity,battery_polarity,display_polarity,storage_polarity,camera_polarity
0,[screen easily worthy device ipad users want t...,"[1, 0, -1, 0, 0, 0, -1]",[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0]
1,[would pretty embarrasing use tablet purposes ...,"[1, 1, 1, -1, -1, -1, 1, -1]","[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]"
2,"[indeed charging pot complet, bought phone dec...","[0, 1, -1, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]"
3,"[edge screen fell part part, excellent phone c...","[0, 1, 1]",[1],"[0, 0, 0, 1, 1]","[1, 1]",[0]
4,[nwith unique looks hdmi fhd screen speakers m...,"[1, 1, 0, 1, 1]","[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0]


In [23]:
#for creating coloume in new_Data DataFrame
url = "Data/all_brands_export.csv"
    # url = "Data/acer-phones_export.csv"
old_data=pd.read_csv(url,low_memory = False,sep =',')


url_1 = "topic_modeling/price_pkr.csv"
price_change=pd.read_csv(url_1,low_memory = False,sep =',')




new_Data = pd.DataFrame()
new_Data["Link"] = old_data["Link"]
new_Data["Name"]=old_data["Name"]
new_Data["storage"] =old_data["Storage"]
new_Data["battery"] =old_data["Battery"]
new_Data["camera"] =old_data["cam1modules"]
new_Data["display"] =old_data["displaytype"]



new_Data["Price"] =price_change["price"]+" "+"PKR"

# new_Data["polarity"] = data["polarity"]
new_Data["battery_polarity"]=data["battery_polarity"]
new_Data["display_polarity"]=data["display_polarity"]
new_Data["storage_polarity"]=data["storage_polarity"]
new_Data["camera_polarity"]=data["camera_polarity"]

In [24]:
new_Data.head()

,Link,Name,storage,battery,camera,display,Price,battery_polarity,display_polarity,storage_polarity,camera_polarity
0,https://www.gsmarena.com/acer_chromebook_tab_1...,Acer Chromebook Tab 10,32GB storage microSDXC,4500 mAh Li-Po,5 MP,IPS LCD,69910 PKR,[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0]
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,Acer Iconia Talk S,16GB/32GB storage microSDXC,3400 mAh Li-Ion,13 MP AF,IPS LCD,36014 PKR,"[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]"
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,Acer Liquid Z6 Plus,32GB storage microSDXC,4080 mAh Li-Po,13 MP AF,IPS LCD,52962 PKR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]"
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,Acer Liquid Z6,8GB storage microSDXC,2000 mAh Li-Ion,8 MP AF,IPS LCD,25421 PKR,[1],"[0, 0, 0, 1, 1]","[1, 1]",[0]
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,Acer Iconia Tab 10 A3-A40,16GB/32GB/64GB storage microSDXC,6100 mAh Li-Ion,5 MP,IPS LCD,48725 PKR,"[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0]


In [25]:
#using averg

def features_sum():
    battery_polarity_sum = []
    storage_polarity_sum = []
    display_polarity_sum = []
    camera_polarity_sum = []
    polarity = []

    for battery_sum in new_Data["battery_polarity"]:
        # print(sum(battery_sum))
        add=sum(battery_sum)
        # res = "{:.2f}".format(add)
        res=add
        battery_polarity_sum.append(res)
    for display_sum in new_Data["display_polarity"]:
        # print(sum(display_sum))
        add=sum(display_sum)
        # res = "{:.2f}".format(add)
        res=add
        display_polarity_sum.append(res)
        
    for storage_sum in new_Data["storage_polarity"]:
        # print(sum(storage_sum))
        add=sum(storage_sum)
        # res = "{:.2f}".format(add)
        res=add
        storage_polarity_sum.append(res)
        
    for camera_sum in new_Data["camera_polarity"]:
        # print(sum(camera_sum))
        add=sum(camera_sum)
        # res = "{:.2f}".format(add)
        res=add
        camera_polarity_sum.append(res)
   

   
        
    new_Data["battery_sum"] = battery_polarity_sum    
    new_Data["display_sum"] = display_polarity_sum    
    new_Data["storage_sum"] = storage_polarity_sum    
    new_Data["camera_sum"]  = camera_polarity_sum 

    
    return new_Data#battery_polarity_sum,display_polarity_sum,storage_polarity_sum,camera_polarity_sum
        

In [26]:
features_sum().head()

,Link,Name,storage,battery,camera,display,Price,battery_polarity,display_polarity,storage_polarity,camera_polarity,battery_sum,display_sum,storage_sum,camera_sum
0,https://www.gsmarena.com/acer_chromebook_tab_1...,Acer Chromebook Tab 10,32GB storage microSDXC,4500 mAh Li-Po,5 MP,IPS LCD,69910 PKR,[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0],0,0,-2,0
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,Acer Iconia Talk S,16GB/32GB storage microSDXC,3400 mAh Li-Ion,13 MP AF,IPS LCD,36014 PKR,"[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]",2,1,4,-3
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,Acer Liquid Z6 Plus,32GB storage microSDXC,4080 mAh Li-Po,13 MP AF,IPS LCD,52962 PKR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]",0,1,1,-1
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,Acer Liquid Z6,8GB storage microSDXC,2000 mAh Li-Ion,8 MP AF,IPS LCD,25421 PKR,[1],"[0, 0, 0, 1, 1]","[1, 1]",[0],1,2,2,0
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,Acer Iconia Tab 10 A3-A40,16GB/32GB/64GB storage microSDXC,6100 mAh Li-Ion,5 MP,IPS LCD,48725 PKR,"[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0],2,3,3,0


After Sentiment Analysis


In [27]:
#for convert into file 
new_Data.to_csv('Data_set.csv', mode='w', index=False)

# # print message
print("Data appended successfully.")

Data appended successfully.
